In [43]:
import numpy as np
import pandas as pd
import math
import sys
import datetime as dt

In [2]:
# quarterly recipe data
data = pd.read_csv("../data/split data/2-SP500 Fundamental.csv") 
data_tmp = pd.DataFrame(data)
print(data_tmp.head())
print(data_tmp.tail())
print(data_tmp.shape[0])

   gvkey  datadate  fyearq  fqtr   tic                        conm datacqtr  \
0   1013  19900131    1990     1  ADCT  ADC TELECOMMUNICATIONS INC   1989Q4   
1   1013  19900430    1990     2  ADCT  ADC TELECOMMUNICATIONS INC   1990Q1   
2   1013  19900731    1990     3  ADCT  ADC TELECOMMUNICATIONS INC   1990Q2   
3   1013  19901031    1990     4  ADCT  ADC TELECOMMUNICATIONS INC   1990Q3   
4   1013  19910131    1991     1  ADCT  ADC TELECOMMUNICATIONS INC   1990Q4   

  datafqtr         rdq     actq   ...     oiadpq   revtq   saleq  teqq  \
0   1990Q1  19900223.0   85.600   ...      9.531  62.043  62.043   NaN   
1   1990Q2  19900529.0   87.438   ...      7.588  60.195  60.195   NaN   
2   1990Q3  19900829.0   89.083   ...      9.577  65.814  65.814   NaN   
3   1990Q4  19901213.0  102.525   ...     11.049  71.750  71.750   NaN   
4   1991Q1  19910221.0  107.228   ...     11.012  72.902  72.902   NaN   

    wcapq   xoprq  costat  mkvaltq   prccq  gsector  
0  58.483  49.240       I 

In [3]:
# stock name
stock_name = list(set(data_tmp['tic']))
for i in range(len(stock_name)-1):
    if(type(stock_name[i]) != str):
        del stock_name[i]  # delete nan in the list

"""
# transfer index into date
date = list(data_tmp['datadate'])
a = date
for i in range(len(date)):
    b = dt.strptime(str(date[i]), '%Y%m%d')
    a[i] = b.strftime('%Y-%m-%d')
series = data_tmp
series.index = a
print(series.head())
"""
print(data_tmp.columns.values)

['gvkey' 'datadate' 'fyearq' 'fqtr' 'tic' 'conm' 'datacqtr' 'datafqtr'
 'rdq' 'actq' 'apq' 'atq' 'ceqq' 'chq' 'cogsq' 'cshoq' 'dlcq' 'dlttq' 'dpq'
 'epspxq' 'invfgq' 'lctq' 'lltq' 'lseq' 'msaq' 'niq' 'oiadpq' 'revtq'
 'saleq' 'teqq' 'wcapq' 'xoprq' 'costat' 'mkvaltq' 'prccq' 'gsector']


In [6]:
# daily close price
data_fd = pd.read_csv("../data/split data/1-SP500 Price 1990 to 1999.csv", low_memory = False) # first decade
data_sd = pd.read_csv("../data/split data/1-SP500 Price 2000 to 2009.csv", low_memory = False) # second decade
data_td = pd.read_csv("../data/split data/1-SP500 Price 2010 to 2017.csv", low_memory = False) # third decade

data_fd = pd.DataFrame(data_fd[list(['datadate', 'tic', 'prccd', 'gsector'])])
data_sd = pd.DataFrame(data_sd[list(['datadate', 'tic', 'prccd', 'gsector'])])
data_td = pd.DataFrame(data_td[list(['datadate', 'tic', 'prccd', 'gsector'])])

print(data_fd.shape[0] + data_sd.shape[0] + data_td.shape[0])

data_daily = [data_fd, data_sd, data_td]
data_daily = pd.concat(data_daily)

print(data_daily.shape[0])


stock_name_daily = list(set(data_daily['tic']))
for i in range(len(stock_name_daily)-1):
    if(type(stock_name_daily[i]) != str):
        del stock_name_daily[i]  # delete nan in the list

6426271
6426271


In [136]:
# use trade date instead of report quarterly date
times = list(data_tmp['datadate']) # quarterly report date
for i in range(len(times)):
    quarter = (times[i] - int(times[i]/10000)*10000)
    if 1201 < quarter:
        times[i] = int(times[i]/10000 + 1)*10000 + 301
    if quarter <= 301:
        times[i] = int(times[i]/10000)*10000 + 301
    if 301 < quarter <= 601:
        times[i] = int(times[i]/10000)*10000 + 601
    if 601 < quarter <= 901:
        times[i] = int(times[i]/10000)*10000 + 901
    if 901 < quarter <= 1201:
        times[i] = int(times[i]/10000)*10000 + 1201
    time_tmp = times[i]
    year = int(time_tmp/10000)
    month = int((time_tmp - year*10000)/100)
    day = int(time_tmp - year*10000 - month*100)
    if(time_tmp < 20171114):
        while(dt.date(year, month, day).weekday() > 4):
            time_tmp = time_tmp + 1
            year = int(time_tmp/10000)
            month = int((time_tmp - year*10000)/100)
            day = int(time_tmp - year*10000 - month*100)
        times[i] = time_tmp

data_factor = pd.DataFrame({'datadate': times, 'tic': data_tmp['tic'], 'gsector':data_tmp['gsector']})
print(data_factor.head(16))
print(data_factor.tail(16))

    datadate  gsector   tic
0   19900301     45.0  ADCT
1   19900601     45.0  ADCT
2   19900903     45.0  ADCT
3   19901203     45.0  ADCT
4   19910301     45.0  ADCT
5   19910603     45.0  ADCT
6   19910902     45.0  ADCT
7   19911202     45.0  ADCT
8   19920302     45.0  ADCT
9   19920601     45.0  ADCT
10  19920901     45.0  ADCT
11  19921201     45.0  ADCT
12  19930301     45.0  ADCT
13  19930601     45.0  ADCT
14  19930901     45.0  ADCT
15  19931201     45.0  ADCT
       datadate  gsector   tic
91200  20140303     20.0  ALLE
91201  20140602     20.0  ALLE
91202  20140901     20.0  ALLE
91203  20141201     20.0  ALLE
91204  20150302     20.0  ALLE
91205  20150601     20.0  ALLE
91206  20150901     20.0  ALLE
91207  20151201     20.0  ALLE
91208  20160301     20.0  ALLE
91209  20160601     20.0  ALLE
91210  20160901     20.0  ALLE
91211  20161201     20.0  ALLE
91212  20170301     20.0  ALLE
91213  20170601     20.0  ALLE
91214  20170901     20.0  ALLE
91215  20171201     20.0  AL

In [137]:
data_factor['prccd'] = [0]*data_factor.shape[0]
for i in stock_name:
    if (i in stock_name_daily):
        ind_quarter = np.where(data_factor['tic'] == i)[0]
        date_price_daily = data_daily.loc[data_daily['tic'] == i, list(['datadate', 'prccd'])]
        date_price_daily.index = range(date_price_daily.shape[0])
        date_max = max(date_price_daily['datadate'])
        date_min = min(date_price_daily['datadate'])
        for j in ind_quarter:
            if (data_factor.loc[j, 'datadate'] < date_min or data_factor.loc[j, 'datadate'] > date_max):
                data_factor.loc[j, 'prccd'] = np.nan
            else:
                if(data_factor.loc[j, 'datadate'] in set(date_price_daily['datadate'])):
                    ind_tmp = np.where(date_price_daily['datadate'] == data_factor.loc[j, 'datadate'])[0]
                    data_factor.loc[j, 'prccd'] = date_price_daily.loc[ind_tmp, 'prccd'].values
                else:
                    data_factor.loc[j, 'prccd'] = np.nan
 

In [138]:
print(data_factor)
writer = pd.ExcelWriter('trade date and price.xlsx')
data_factor.to_excel(writer,'Sheet1')
writer.save()

       datadate  gsector   tic   prccd
0      19900301     45.0  ADCT  20.750
1      19900601     45.0  ADCT  22.000
2      19900903     45.0  ADCT     NaN
3      19901203     45.0  ADCT  23.625
4      19910301     45.0  ADCT  32.000
5      19910603     45.0  ADCT  40.750
6      19910902     45.0  ADCT     NaN
7      19911202     45.0  ADCT  25.125
8      19920302     45.0  ADCT  25.750
9      19920601     45.0  ADCT  35.000
10     19920901     45.0  ADCT  36.000
11     19921201     45.0  ADCT  45.500
12     19930301     45.0  ADCT  42.000
13     19930601     45.0  ADCT  54.750
14     19930901     45.0  ADCT  36.000
15     19931201     45.0  ADCT  35.375
16     19940301     45.0  ADCT  36.875
17     19940601     45.0  ADCT  41.375
18     19940901     45.0  ADCT  46.000
19     19941201     45.0  ADCT  41.000
20     19950301     45.0  ADCT  29.500
21     19950601     45.0  ADCT  30.250
22     19950901     45.0  ADCT  41.750
23     19951201     45.0  ADCT  43.125
24     19960301     45.0 

In [139]:
# factors calculation function
def revenue_growth(df):
    # REVTQ (T) / REVTQ (T-1) -1
    rev = df['revtq']
    rev = round(rev.shift(1)/rev - 1, 3)
    return rev

def price_to_cash_flow_ratio(df2, df):
    # Adjusted Close Price * CSHOQ 
    # / [(NIQ (Net Income (Loss)) + DPQ (Depreciation and Amortization, Total) – (WCAPQ (T)- WCAPQ (T-1))]
    tmp = df['wcapq']
    tmp = round(tmp.shift(1) - tmp, 3)
    ptcfr = round(df2['prccd']*df['cshoq']/(df['niq'] + df['dpq'] - tmp), 3)
    return ptcfr
    
    
def cash_ratio(df):
    # [CHQ (Cash) + MSAQ (Marketable Security Adjustments)] / LCTQ (Current Liabilities, Total)
    chq_ratio = round((df['chq'] + df['msaq'])/df['lctq'], 3)
    return chq_ratio

def ebitda(df):
    # EV = MKVALTQ (Market Value, Total) + DLTTQ (Long-Term Debt, Total) 
    # + DLCQ (Debt in Current Liabilities) - CHQ (Cash)
    ev = round(df['mkvaltq'] + df['dlttq'] + df['dlcq'] - df['chq'], 3)
    
    # EBITDA = REVTQ (Revenue, Total) – XOPRQ (Operating Expense, Total) 
    # + DPQ (Depreciation and Amortization, Total)
    ebitda = round(df['revtq'] - df['xoprq'] + df['dpq'], 3)
    return ebitda

def ev_to_cfo(df):
    # [MKVALTQ (Market Value, Total) + DLTTQ (Long-Term Debt, Total) + DLCQ (Debt in Current Liabilities) - CHQ (Cash)]
    # /[(NIQ (Net Income (Loss)) + DPQ (Depreciation and Amortization, Total) – (WCAPQ (T)- WCAPQ (T-1))]
    tmp = df['wcapq']
    tmp = tmp.shift(1) - tmp
    
    num = df['mkvaltq'] + df['dlttq'] + df['dlcq'] - df['chq']
    num2 = df['niq'] + df['dpq'] - tmp
    evtocfo = round(num/num2, 3)
    return evtocfo


In [140]:
# factors calculation
pd.set_option('mode.chained_assignment', None)
data_factor_store = pd.DataFrame(columns = ['datadate', 'gsector', 'tic', 'prccd',
                                            'revenue_growth', 'price_to_cash_flow_ratio', 
                                            'cash_ratio', 'ebitda', 'ev_to_cfo'])
for i in stock_name:
    df = data_tmp.loc[data_tmp['tic'] == i, ]
    df2 = data_factor.loc[data_factor['tic'] == i, ]
    df2.loc[:, 'revenue_growth'] = list(revenue_growth(df))
    df2.loc[:, 'price_to_cash_flow_ratio'] = list(price_to_cash_flow_ratio(df2, df))
    df2.loc[:, 'cash_ratio'] = list(cash_ratio(df))
    df2.loc[:, 'ebitda'] = list(ebitda(df))
    df2.loc[:, 'ev_to_cfo'] = list(ev_to_cfo(df))
    data_factor_store = [data_factor_store, df2]
    data_factor_store = pd.concat(data_factor_store)    

In [141]:
data_factor = data_factor_store
print(data_factor)
writer = pd.ExcelWriter('factor matrix with tradable date.xlsx')
data_factor.to_excel(writer,'Sheet1')
writer.save()

       datadate  gsector    tic   prccd  revenue_growth  \
1501   19900601     50.0   AT.2  34.250             NaN   
1502   19900903     50.0   AT.2     NaN          -0.010   
1503   19901203     50.0   AT.2  32.125          -0.022   
1504   19910301     50.0   AT.2  37.875          -0.016   
1505   19910603     50.0   AT.2  41.000          -0.076   
1506   19910902     50.0   AT.2     NaN          -0.060   
1507   19911202     50.0   AT.2  35.000          -0.053   
1508   19920302     50.0   AT.2  37.875          -0.002   
1509   19920601     50.0   AT.2  39.000          -0.028   
1510   19920901     50.0   AT.2  41.750          -0.026   
1511   19921201     50.0   AT.2  47.875          -0.009   
1512   19930301     50.0   AT.2  47.750          -0.018   
1513   19930601     50.0   AT.2  51.375          -0.024   
1514   19930901     50.0   AT.2  30.500          -0.039   
1515   19931201     50.0   AT.2  27.250          -0.005   
1516   19940301     50.0   AT.2  27.250          -0.127 